In [1]:
import arcpy

In [2]:
arcpy.env.overwriteOutput = True

In [ ]:
#checks out the extension from the licence to get the network extension to create networks
arcpy.CheckOutExtension("network")

In [3]:
#makes a folder variable based off the path you give. It needs to have all of the appropriate data in it.
folder = r"C:\Users\casiegle/OneDrive - University of Iowa/Desktop/dc_data"

In [4]:
#Creates a roads_shp variable using the folder path plus the path to the shape file.
roads_shp = folder + "/roads/roads.shp"

In [5]:
#creates a describe object for later use
desc = arcpy.Describe(roads_shp)

In [7]:
#prints the spatial reference name so you can check to see later on if it is the same as your points for the network
print(desc.spatialReference.name)

GCS_WGS_1984


In [9]:
#creates a feature dataset based off of the spatial reference of the roads shapefile
arcpy.CreateFeatureDataset_management(arcpy.env.workspace, "featuredataset", desc.spatialReference)

<Result 'C:\\Users\\casiegle\\OneDrive - University of Iowa\\Desktop\\DCProject\\DCProject.gdb\\featuredataset'>

In [10]:
#copies features from the roads shapefile into the "featuredataset" we just created
arcpy.CopyFeatures_management(roads_shp, "featuredataset/roads")

<Result 'C:\\Users\\casiegle\\OneDrive - University of Iowa\\Desktop\\DCProject\\DCProject.gdb\\featuredataset\\roads'>

In [13]:
#Creates a firestations_shp variable using the folder path plus the path to the shape file.
firestations_shp = folder + "/FireStations/Fire_Stations.shp"

In [14]:
#prints the spatial reference name so you can check to see later on if it is the same as your points for the network
#you need to make sure this is the same as the roads.shp file reference name
print(arcpy.Describe(firestations_shp).spatialReference.name)

GCS_WGS_1984


In [15]:
#copy the features from the firestations shapefile into the featuredataset we created that has the roads as well
arcpy.CopyFeatures_management(firestations_shp, "featuredataset/firestations")

<Result 'C:\\Users\\casiegle\\OneDrive - University of Iowa\\Desktop\\DCProject\\DCProject.gdb\\featuredataset\\firestations'>

In [16]:
#this creates the actual network dataset with the roads shapefile. it outputs a network dataset
arcpy.na.CreateNetworkDataset("featuredataset", "roads_ND", ["roads"])

<Result 'C:\\Users\\casiegle\\OneDrive - University of Iowa\\Desktop\\DCProject\\DCProject.gdb\\featuredataset\\roads_ND'>

In [17]:
#this builds the network based off of the roads network dataset we just created in the previous cell
arcpy.BuildNetwork_na("featuredataset/roads_ND")

<Result 'C:\\Users\\casiegle\\OneDrive - University of Iowa\\Desktop\\DCProject\\DCProject.gdb\\featuredataset/roads_ND'>

In [25]:
#this creates the service area analysis layer based off the network dataset, names it myArea,
#with 300, 600, and 1500 meter cutoffs with high polygon detail and overlapping, with rings for easier visuals
area = arcpy.na.MakeServiceAreaAnalysisLayer("featuredataset/roads_ND", "myArea", "Length", "FROM_FACILITIES",  [300, 600, 1500], polygon_detail="HIGH", geometry_at_overlaps="OVERLAP", geometry_at_cutoffs = "RINGS")

In [26]:
#this gets the output from the service area
areaLayer = area.getOutput(0)

In [27]:
#creates the classes from the output
na_classes = arcpy.na.GetNAClassNames(areaLayer, "INPUT")

In [28]:
#prints the classes 
print(na_classes)

{'Facilities': 'Facilities', 'Barriers': 'Point Barriers', 'PolylineBarriers': 'Line Barriers', 'PolygonBarriers': 'Polygon Barriers'}


In [29]:
#creates a sublayer variable as the facilities from the classes
sublayer = na_classes["Facilities"]

In [30]:
#this adds the locations to the area layer which are the firestations
arcpy.na.AddLocations(areaLayer, sublayer, "firestations", "", "")

<Result 'myArea'>

In [31]:
#this actually solves the service area analysis and shows it to you on the map 
#it outputs a layer you can export different things out of
arcpy.na.Solve(areaLayer)

id,value
0,a Layer object
1,true
